In [31]:
import bs4
import requests
import pandas
import re
import math

### Fetching page numbers

In [35]:
url='https://www.naukri.com/machine-learning-jobs-'
fhand=requests.get(url)
soup=bs4.BeautifulSoup(fhand.text,'html.parser')

pages=soup.find('span',{'class':'cnt'}).text
pages=(re.search('1-([0-9]+) of ([0-9]+)',pages)) #1-50 of 4870
pages=int(pages.group(2))/int(pages.group(1))
pages=math.ceil(pages)
print(pages)

98


### Fetch all URLS and download pages

In [41]:
urlList=list()
soupList=list()
pages=1
for i in range(1,pages+1): # from 1 to 98
    urlList.append(url+str(i))
    fhand=requests.get(urlList[i-1])# since i starts from 1
    soup=bs4.BeautifulSoup(fhand.text,'html.parser')
    soupList.append(soup)

### Scrape data from pages

In [42]:

companyList=list()
for soup in soupList:
    cards=soup.find_all('div',{"type": "tuple"})
    for card in cards:
        data=list()

        ########## CARD Information ##########

        title=card.find('li',{'class':'desig'}).get('title')
        location=card.find('span',{'class':'loc'}).find('span').contents[0] 
        company=card.find('span',{'class':'org'}).contents[0]
        desc=card.find('span',{'class':'desc'})
        if(desc != None): desc = desc.contents[0]
        exp=card.find('span',{'class':'exp'})
        if(exp != None): exp = exp.contents[1]

        ########## Job Link ##########
        link=(card.find('a',{'class':'content'}).get('href'))
        linkedPage=requests.get(link)
        linkedPageSoup=bs4.BeautifulSoup(linkedPage.text,'html.parser')

        ksTags=linkedPageSoup.find('div',{'class':'ksTags'})
        if(ksTags!=None):
            ksTags=ksTags.find_all('font',{'class':'hlite'})
            ksTags=(str(ksTags).replace('<font class="hlite"> ','').replace('</font>','').replace('[','').replace(']',''))


        ########## Details ##########

        details=linkedPageSoup.find('div',{'class':'jDisc mt20'})
        detailColValues=list()
        if(details!=None):
            for tag in details.find_all('span'):
                detailColValues.append(tag.text)

        ########## Educational Details ##########

        details=linkedPageSoup.find('div',{'class':'jDisc mt10 edu'})
        if(details!=None):
            for tag in details.find_all('em'):
                detailColValues.append(tag.findNext().text)

        data.append(title)
        data.append(company)
        data.append(location)
        data.append(desc)
        data.append(link)
        data.append(ksTags)
        data.append(exp)
        data.extend(detailColValues)

        companyList.append(data)

    

### Store above data in Dataframe as well as CSV

In [40]:
colNames=['Title','Company','Location','Description','Link','KeySkills','Experience']
detailColNames=['Salary', 'Industry', 'Functional Area', 'Role Category', 'Role', 'EmploymentType', ' UG', 'PG','Doctorate']
colNames.extend(detailColNames)

df=pandas.DataFrame(companyList,columns=colNames)
df.to_csv('naukri1000.csv')
display(df)

,Title,Company,Location,Description,Link,KeySkills,Experience,Salary,Industry,Functional Area,Role Category,Role,EmploymentType,UG,PG,Doctorate
0,Senior Lead - Artificial Intelligence/machine ...,CenturyLink Technologies India Pvt Ltd,Bengaluru,None,https://www.naukri.com/job-listings-Senior-Lea...,"Deep Learning, Machine Learning, Data Mining, ...",12-16 yrs,Not Disclosed by Recruiter,IT-Software / Software Services,"IT Software - Application Programming , ...",Programming & Design,Team Lead/Technical Lead,"Permanent Job, Full Time",B.Tech/B.E. - Any Specialization,"M.Tech - Any Specialization, MCA - Computers",None
1,Machine Learning/nlp Experts (location Chakala...,Taglr Technologies Private Limited,"Mumbai, Mumbai Suburbs",- 3-5 yrs of experience with ML tools and libr...,https://www.naukri.com/job-listings-Machine-Le...,"Deep Learning, Computer Vision, Image Processi...",4-9 yrs,Not Disclosed by Recruiter,Internet / Ecommerce,"IT Software - Application Programming , ...",System Design/Implementation/ERP/CRM,Solution Architect / Enterprise Architect,"Permanent Job, Full Time",None,None,None
2,Data Scientist - Machine Learning,Volkswagen IT Services India Pvt. Ltd,Pune,Following qualifications and competencies are ...,https://www.naukri.com/job-listings-Data-Scien...,"machine learning, natural language processing,...",3-8 yrs,Not Disclosed by Recruiter,IT-Software / Software Services,"IT Software - Application Programming , ...",Programming & Design,Software Developer,"Permanent Job, Full Time",None,None,None
3,Team Lead - Artificial Intelligence/machine Le...,Axcelovate,Bengaluru,- 5-8 years of relevant work experience;- Good...,https://www.naukri.com/job-listings-Team-Lead-...,"Machine Learning, NLP, Artificial Intelligence...",5-10 yrs,Not Disclosed by Recruiter,IT-Software / Software Services,"IT Software - Application Programming , ...",Programming & Design,Team Lead/Technical Lead,"Permanent Job, Full Time",Any Graduate - Any Specialization,Any Postgraduate - Any Specialization,Doctorate Not Required
4,"Senior Analyst - Spss, R, Machine Learning",Vodafone India Limited,"Mumbai, Pune",The primary purpose of this role is to deliver...,https://www.naukri.com/job-listings-Senior-Ana...,None,3-8 yrs,None,None,None,None,None,None,None,None,None
5,Analytics - Quantitative Manager - Machine Lea...,Silverpeople,Mumbai,- Should have strong previous work experience ...,https://www.naukri.com/job-listings-Analytics-...,"Market Risk, Credit Risk, Finance, Structured ...",5-9 yrs,Not Disclosed by Recruiter,IT-Software / Software Services,Analytics & Business Intelligence,Analytics & BI,Analytics Manager,"Permanent Job, Full Time",Any Graduate - Any Specialization,Any Postgraduate - Any Specialization,Ph.D
6,Machine Learning Engineer - Python,Catalyst,Bengaluru,- Experience in building predictive maintenanc...,https://www.naukri.com/job-listings-Machine-Le...,"Data Science, Machine Learning, Python, Predic...",2-7 yrs,Not Disclosed by Recruiter,IT-Software / Software Services,"IT Software - Application Programming , ...",Admin/Maintenance/Security/Datawarehousing,System Administrator,"Permanent Job, Full Time",Any Graduate - Any Specialization,Any Postgraduate - Any Specialization,Doctorate Not Required
7,Machine Learning,Accenture,Bengaluru,e: Should have expertise in advanced analytics...,https://www.naukri.com/job-listings-Machine-Le...,"Python, Data mining, Machine learning, Busines...",3-8 yrs,Not Disclosed by Recruiter,IT-Software / Software Services,"IT Software - Application Programming , ...",Programming & Design,Software Developer,"Permanent Job, Full Time",B.Tech/B.E. - Computers,M.Tech - Computers,None
8,Machine Learning,Accenture,Mumbai,e: Should have expertise in advanced analytics...,https://www.naukri.com/job-listings-Machine-Le...,"Python, Data mining, Machine learning, Busines...",2-6 yrs,Not Disclosed by Recruiter,IT-Software / Software Services,"IT Software - Application Programming , ...",Programming & Design,So